Split bounding boxes into new images

In [5]:
import cv2
import json
import os

# Load annotations
with open("Manual_Cusp_Boxes.json") as f:
    data = json.load(f)

image_dir = "/Users/rfalcao/Documents/FYP/ClearCuspsCroppedRotated"
output_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentation"

os.makedirs(output_dir, exist_ok=True)

# Process each image
for img_info in data["images"]:
    img_path = os.path.join(image_dir, img_info["file_name"])
    img = cv2.imread(img_path)
    # Find annotations for this image
    for ann in data["annotations"]:
        if ann["image_id"] == img_info["id"]:
            x, y, w, h = ann["bbox"]  # COCO format: x, y, width, height
            cropped = img[int(y):int(y+h), int(x):int(x+w)]
            
            # Save cropped region
            save_path = os.path.join(output_dir, f"cusp_{ann['id']}.jpg")
            cv2.imwrite(save_path, cropped)


Separate Images into new categories

In [2]:
import os
import json
import shutil

# Paths
image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentation"  # Folder containing all images
annotation_file = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/instances_default.json"  # Exported JSON file
output_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns"

# Create output folders
os.makedirs(f"{output_dir}/cusp_images", exist_ok=True)
os.makedirs(f"{output_dir}/non_cusp_images", exist_ok=True)
os.makedirs(f"{output_dir}/unknown_images", exist_ok=True)

# Load annotation file
with open(annotation_file) as f:
    data = json.load(f)

# Get images with annotations
annotated_images = {}
for ann in data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in annotated_images:
        annotated_images[img_id] = []
    annotated_images[img_id].append(ann["segmentation"])

# Get image file names
image_info = {img["id"]: img["file_name"] for img in data["images"]}

# Process each image
for img_id, filename in image_info.items():
    src_path = os.path.join(image_dir, filename)
    if os.path.exists(src_path):
        if img_id in annotated_images and annotated_images[img_id]:  # Has annotations
            dest_folder = "cusp_images"
        else:  # No annotations
            dest_folder = "non_cusp_images"
        shutil.move(src_path, os.path.join(output_dir, dest_folder, filename))
    else:
        print(f"Warning: {filename} is missing from {image_dir}")

# Move any leftover images (not in JSON)
for filename in os.listdir(image_dir):
    src_path = os.path.join(image_dir, filename)
    dest_path = os.path.join(output_dir, "unknown_images", filename)
    shutil.move(src_path, dest_path)

print("Images sorted successfully!")


Images sorted successfully!


In [8]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import os

# Paths
annotation_file = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/instances_default.json"
image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/cusp_images"
mask_output_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks"
untagged_image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/non_cusp_images"

os.makedirs(mask_output_dir, exist_ok=True)

# Load COCO JSON
coco = COCO(annotation_file)

# Process images
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    img_filename = img_info["file_name"]

    # Try loading from cusp images folder
    img_path = os.path.join(image_dir, img_filename)
    if not os.path.exists(img_path):
        # If not found, check no-cusp images folder
        img_path = os.path.join(untagged_image_dir, img_filename)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Skip missing images
    if img is None:
        print(f"Skipping {img_filename} (not found in any folder)")
        continue

    # Create an empty mask
    mask = np.zeros(img.shape, dtype=np.uint8)

    # Get annotations (if any)
    ann_ids = coco.getAnnIds(imgIds=img_id)
    if ann_ids:
        anns = coco.loadAnns(ann_ids)
        for ann in anns:
            segmentation = ann["segmentation"]
            for seg in segmentation:
                points = np.array(seg, dtype=np.int32).reshape(-1, 2)
                cv2.fillPoly(mask, [points], 255)  # Fill the polygon mask

    # Save the mask (even if empty, for no-cusp cases)
    mask_filename = os.path.join(mask_output_dir, img_filename)
    cv2.imwrite(mask_filename, mask)

print("Masks extracted and saved!")


loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Masks extracted and saved!


In [9]:
import os
import cv2
import numpy as np

# Paths

padded_image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/images"
padded_mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/masks"

image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/cusp_images"
mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks"

os.makedirs(padded_image_dir, exist_ok=True)
os.makedirs(padded_mask_dir, exist_ok=True)

# Step 1: Find max width & height in dataset
max_width, max_height = 0, 0
for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        h, w = img.shape
        max_width = max(max_width, w)
        max_height = max(max_height, h)

print(f"Max dimensions for padding: {max_width}x{max_height}")

# Step 2: Pad each image & mask
for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    mask_path = os.path.join(mask_dir, filename)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        print(f"Skipping {filename} (missing image or mask)")
        continue

    h, w = img.shape
    pad_top = (max_height - h) // 2
    pad_bottom = max_height - h - pad_top
    pad_left = (max_width - w) // 2
    pad_right = max_width - w - pad_left

    # Apply same padding to image and mask
    padded_img = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)
    padded_mask = cv2.copyMakeBorder(mask, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)

    # Save results
    cv2.imwrite(os.path.join(padded_image_dir, filename), padded_img)
    cv2.imwrite(os.path.join(padded_mask_dir, filename), padded_mask)

print("✅ Padding complete! All cusp images and masks are now the same size.")


Max dimensions for padding: 870x851
✅ Padding complete! All cusp images and masks are now the same size.
